# Dimensionaliteitsreductie

@author: Jan Verwaeren - Arne Deloose

@course: Machine learning: van theorie tot praktijk

Deze notbook bevat enkele illustraties van frequent gebruikte dimensionaliteitsreductietechnieken:
- (kernel) principale componenten analyse
- (metric) Multidimensional scaling
- TSNE
- UMAP


## 1a. ILLUSTRATIE: (kernel) principale componenten analyse (PCA) op iris dataset

******

In deze sectie wordt PCA geïllustreerd op de iris dataset. 

**Inladen nodige libraries en enkele handige visualisatiefuncties**

In [ ]:
import pandas as pd                                # pandas dataframes
import numpy as np                                 # numpy datastructuur voor homogene arrays (matrices)
from sklearn.preprocessing import StandardScaler   # scikit learn: tools voor standaardiseren data (gemiddelde=0 en stdev=1)
from sklearn.decomposition import PCA, KernelPCA   # scikit learn: klasses voor (kernel)PCA
from sklearn.manifold import TSNE                  #TSNE

import matplotlib.pyplot as plt                    # visualisatiebibliotheken
import seaborn as sns

!python -m pip install jellyfish
import jellyfish #lev afstand

#zelf gedefinieerde functies

def scree_plot(pca_model):
    """ genereert scree plot op basis van een gefit PCA model (argument is PCA-object, na fitten) """
    PC_values = np.arange(pca_model.n_components_) + 1
    fig, ax = plt.subplots()
    plt.plot(PC_values, pca_model.explained_variance_ratio_, 'ro-', linewidth=2)
    plt.title('Scree Plot')
    plt.xlabel('Principal Component')
    plt.ylabel('Proportion of Variance Explained')
    return(fig)

def biplot_func(score, pca_model, labels=None, y=None, show_index=False):
    """ genereert biplot, met argumenten:
         - score: de scores (twee eerste PC) die men wenst te visualiseren
         - pca_model: PCA-object, na fitten
         - labels: labels van PCs die als naam worden weergegeven
         - y: numerical vector, length equal to the number of observations used for color-coding the datapoints
         - show_index: geeft aan of de indices van de objecten bij de scores worden getoond
    """
    coeff=np.transpose(pca_model.components_[0:2, :]).copy()
    xs = score[:, 0]
    ys = score[:, 1]
    n = coeff.shape[0]
    coeff[:, 0] = coeff[:,0]*xs.std()*np.sqrt(len(xs))
    coeff[:, 1] = coeff[:,1]*ys.std()*np.sqrt(len(ys))
    fig, ax = plt.subplots()
    im = ax.scatter(xs, ys, c = y) #use y as color
    if show_index:
        for i in range(len(xs)):
            ax.text(xs[i], ys[i], i+1)
    if not y is None:
        fig.colorbar(im, ax=ax)
    for i in range(n):
        ax.arrow(0, 0, coeff[i, 0], coeff[i,1], color = 'r',alpha = 0.5)
        if labels is None:
            ax.text(1.1*coeff[i,0], 1.1*coeff[i,1], "Var"+str(i+1), color = 'g', ha = 'center', va = 'center')
        else:
            ax.text(1.1*coeff[i,0], 1.1*coeff[i,1], labels[i], color = 'g', ha = 'center', va = 'center')           
    ax.set_xlim(1.2*min(np.min(coeff[:, 0]), np.min(xs)), 1.2*max(np.max(coeff[:, 0]), np.max(xs)))
    ax.set_ylim(1.2*min(np.min(coeff[:, 1]), np.min(ys)), 1.2*max(np.max(coeff[:, 1]), np.max(ys)))
    ax.set_xlabel("PC{}".format(1))
    ax.set_ylabel("PC{}".format(2))
    ax.grid()
    return(fig, ax)

**STAP 1: Inladen iris dataset**

In [ ]:
url='https://raw.githubusercontent.com/jverwaer/ugain_machine_learning/main/Data/iris.csv' #raw github link
df=pd.read_csv(url, sep = ";")
df.head()

**STAP 2: Extraheren numerieke variabelen en schalen van de data**

In [ ]:
X = np.array(df.iloc[:,:4])
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)      # gemiddelde 0 en stdev 1 voor elke variabele

**STAP 3: Toepassen PCA**

In [ ]:
pca_model = PCA()                   # genereren PCA object
x_new = pca_model.fit_transform(X)  # berekenen scores en loadings (x_new bevat de scores, de loadings kan je terugvinden
                                    # in pca_model.components_)

**STAP 4: Visualiseren scores**

In [ ]:
# PCA scores en soort-labels samenvatten in een pandas dataframe
df_pca = pd.DataFrame(x_new[:, 0:2], columns = ["PC1", "PC2"])
df_pca["soort"] = df["soort"]

# visualisatie met seaborn
sns.relplot(df_pca,
            x='PC1',
            y='PC2',
            hue='soort')

**STAP 5: bekijken loadings**

In [ ]:
# wrappen van loadings en variabelenamen in dataframe
df_pca_loadings = pd.DataFrame(np.transpose(pca_model.components_[0:2, :]),
                               columns = ["PC1", "PC2"])
df_pca_loadings["variabelen"] = df.columns[:-1]
df_pca_loadings

In [ ]:
# visualiseren loadings
ax = sns.scatterplot(df_pca_loadings,
                 x='PC1',
                 y='PC2')

#voeg labels toe
def plotlabel(xvar, yvar, label):
    ax.text(xvar+0.002, yvar, label)

# use lambda function to apply it to the scatterplot (shortcut for a for-loop):
dummy = df_pca_loadings.apply(lambda x: plotlabel(x['PC1'],  x['PC2'], x['variabelen']), axis=1)

**STAP 6: combineren scores en loadings op 1 plot: de biplot**

In [ ]:
# optional: recode 'soort' naar numerieke vector, met 'setosa' = 0, 'versicolor' = 1, 'virginica' = 2
soort_recoded = np.unique(df["soort"], return_inverse = True)[1]

[fig, ax] = biplot_func(x_new[:, 0:2],
                        pca_model,
                        labels=df.columns[:-1],
                        y = soort_recoded,
                        show_index=False)
fig

**STAP 7: bekijken verklaarde variabiliteit per PC**

In [ ]:
fig = scree_plot(pca_model)
fig

## 1b. ILLUSTRATIE: (kernel) principale componenten analyse (PCA) op olympic dataset

******

In deze sectie wordt PCA geïllustreerd op de olympic dataset. 

**Stap 1: inladen dataset**

In [ ]:
url = 'https://raw.githubusercontent.com/jverwaer/ugain_machine_learning/main/Data/olympic.csv' #raw github link
df=pd.read_csv(url)
df.head()

**Stap 2 + 3: Extraheren variabelen en toepassen PCA**

In [ ]:
#PCA
X=df.loc[:, 'm100':'m1500']
y=df.loc[:, 'score']

# scale data and fit pca
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)    
pca_model = PCA()
x_new = pca_model.fit_transform(X)

**Stap 4-6: biplot**

In [ ]:
#Call the function with 2 pcs
[fig, ax] = biplot_func(x_new[:, 0:2], pca_model, labels=df.loc[:, 'm100':'m1500'].columns, y=df['score'], show_index=True)
fig

**Stap 7: variantie (scree plot)**

In [ ]:
scree_plot(pca_model)

## 2. Oefeningen PCA

********

**1. Zware metalen in de Schelde**

Aan het departement omgeving onderzochten enkele onderzoekers de vervuiling van grasland
in de nabijheid van de rivier de Schelde. De concentratie van acht zware metalen (cadmium,
chroom, koper, ijzer, mangaan, nikkel, lood, zink) werd gemeten op 19 verschillende locaties
elke keer op een diepte van 5 en 20 cm. De nabijheid van de locatie tot de rivier werd genoteerd
als ver(0) of dicht(1). Je vindt de data terug in **heavymetals.csv**.

**1.1. Lees de data in en bestudeer ze zodat je er voeling mee krijgt.**

In [ ]:
url='https://raw.githubusercontent.com/jverwaer/ugain_machine_learning/main/Data/heavymetals.csv'

df=pd.read_csv(url)
df.head()

**1.2. Voer een PCA-analyse uit. Schaal je de data of niet? Hoeveel componenten zou je behouden?**

Data schalen kan je doen via:

- `scaler = StandardScaler()`

- `scaler.fit(X)`

- `X=scaler.transform(X)`


In [ ]:
X=df.loc[:, ***]
y=df.loc[:, ***]

#schalen?
*** 

#PCA fitten
pca_model = PCA()
x_new = pca_model.fit_transform(X)

#hoeveel componenten heb je nodig?
***

**1.3. Maak een biplot en interpreteer deze. Kleur de locaties in de biplot volgens de nabijheid bij de rivier.**

In [ ]:
# 1.3. biplot
[fig, ax] = biplot_func(***, ***, 
                        labels=df.loc[:, 'Cd5':'Zn20'].columns,  y=y)
fig

**2. Gestrande dolfijnen**

Van een aantal gestrande dolfijnen aan de Schotse kusten, heeft men de vetzurensamenstelling
van hun vetlaag bestudeerd. De samenstelling - en dan vooral de hoeveelheid kortere versus
langere en onverzadigde versus verzadigde vetzuren - kan namelijk een indicatie zijn voor het
dieet van de dolfijnen. Voor meer dan 20 vetzuren werd de aanwezige concentratie bepaald. Je
kan de data terugvinden in **dolfijnen.csv**.

Vetzuren zijn genoteerd als FAxx.yy. De xx staat voor het aantal koolstofatomen in de vetzuurketen. De yy geeft het aantal dubbele bindingen weer. Verzadigde vetzuren hebben er geen,
dus daar is yy gelijk aan nul. Onverzadigde vetzuren worden genoteerd als an.b, waarbij a
het aantal dubbele bindingen geeft, en b het eerste koolstofatoom waar een dubbele binding
voorkomt (telkens te tellen van het uiteinde, niet de -COOH kop).
Dus bvb FA20.4n.3 is een onverzadigd vetzuur met een koolstofketen van 20 koolstofatomen
en 4 dubbele bindingen (telkens gescheiden door een CH2) die beginnen op positie 3.

**2.1. Lees de data in en bestudeer ze zodat je er voeling mee krijgt.**

In [ ]:
url='https://raw.githubusercontent.com/jverwaer/ugain_machine_learning/main/Data/dolfijnen.csv'

df=pd.read_csv(url)
df.head()

**2.2. Voer een PCA-analyse uit. Hoeveel dimensies zou je behouden om deze data voor te stellen?**

In [ ]:
X=***

# scale data and fit pca
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

#PCA model fitten
***

#hoeveel componenten heb je nodig?
***

**2.3. Interpreteer de eerste component. Bekijk hiervoor de nodige diagnostische grafieken voor de eerste twee dimensies. Kijk ook eens naar de loadings voor de verschillende vetzuren.**

In [ ]:
df_pc=pd.DataFrame()
df_pc['PC1']=pca_model.components_[***] #eerste component
df_pc['PC2']=pca_model.components_[***] #tweede component
df_pc['FA']=*** #naam van de vetzuren
df_pc #toon de dataframe

In [ ]:
#plot
ax = sns.scatterplot(df_pc,
                 x='PC1',
                 y='PC2')

#voeg labels toe
def plotlabel(xvar, yvar, label):
    ax.text(xvar+0.002, yvar, label)

# use lambda function to apply it to the scatterplot (shortcut for a for-loop):
dummy = df_pc.apply(lambda x: plotlabel(x['PC1'],  x['PC2'], x['FA']), axis=1)

**2.4. Er bevinden zich 4 dolfijnen in de dataset die doodgeboren zijn, kan je hen identificeren?**

In [ ]:
#data
df_score=pd.DataFrame()
df_score['x1']=*** #eerste component
df_score['x2']=*** #tweede component
df_score['index']=***

In [ ]:
# plot
ax = sns.scatterplot(df_score,
                 x='x1',
                 y='x2')

#voeg labels toe
def plotlabel(xvar, yvar, label):
    ax.text(xvar+0.002, yvar, label)

# use lambda function to apply it to the scatterplot (shortcut for a for-loop):
dummy = df_score.apply(lambda x: plotlabel(x['x1'],  x['x2'], x['index']), axis=1)

**EXTRA: 2.5. herhaal de vraag in 2.4 met een kernel-pca (polynomiale kernel). Bekom je hetzelfde resultaat?**

In [ ]:
#kernel pca
from sklearn.decomposition import KernelPCA

***

## 3. ILLUSTRATIE: Multidimensional scaling (MDS)

*****

In de onderstaande sectie wordt het gebruik van Multdimensional Scaling (MDS geïllustreerd).

**STEP 1**: import libraries

MDS is an example of what is generally known as manifold learning techniques. Therefore, it can be found in the `manifold` submodule of the `sklearn` package. The implementation of MDS in `sklearn` (and in particular the optimization routine behind it) relies on stress majorization, a brief description can be found [here](https://en.wikipedia.org/wiki/Stress_majorization). The documentation page of `sklearn` contains other manifold learning techniques, an overview kan be found [here](https://scikit-learn.org/stable/modules/manifold.html).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import euclidean_distances

**STEP 2**: create distance matrix

A distance matrix is a square symmetric numpy array (dimensions equal the number of objects in the dataset). In the example below, the distance matrix is hand-coded, but in most cases it is the result of some distance computation. In most applications of MDS, choosing a good distance measure (that is relevant to the problem at hand) is key to the succes of MDS. 

In [ ]:
D = np.array([[0, 1, 2, 3],
              [1, 0, 1, 2],
              [2, 1, 0, 2],
              [3, 2, 2, 0]])

**STEP 3**: apply MDS (follows the typical workflow of sklearn). 

We need to provide the MDS model: (1) the number of components to use; and (2) the type of distance matrix to use. For the latter, the default is Euclidean distance, assuming that the data is a traditional table (note that if Euclidean distance is used an a regular data-table such as the iris dataset is provided as input, the result would be almost identical to PCA). In this case, however, we want to provide a precomputed distance matrix.

In [ ]:
MDS_model = MDS(n_components=2, 
                dissimilarity='precomputed')
MDS_model.fit_transform(D)

**STEP 4**: wrap results in a pandas dataframe

The attribute `embedding_` contains the computed coordinates. In the example below, they are extracted and wrapped into a Pandas dataframe with column names PC1 and PC2. An additional column is added as an identifier (note that this is not stricly necessary as Pandas dataframes automatically assign an index).

In [ ]:
df = pd.DataFrame(MDS_model.embedding_,
                  columns = ["PC1", "PC2"])
df["idx"] = range(1, 5)

**STEP 5**: plot coordinates

In [ ]:
# plot
ax = sns.scatterplot(df,
                 x='PC1',
                 y='PC2')

#voeg labels toe
def plotlabel(xvar, yvar, label):
    ax.text(xvar+0.002, yvar, label)

# use lambda function to apply it to the scatterplot (shortcut for a for-loop):
dummy = df.apply(lambda x: plotlabel(x['PC1'],  x['PC2'], x['idx']), axis=1)

**STEP 6**: look at the stress & do a check

In [ ]:
print("stress: ", MDS_model.stress_)
euclidean_distances(MDS_model.embedding_)

## 4. Oefeningen MDS


*******

**4.1. Vergelijking van IGHV eiwitten**

Immunoglobulin Heavy Chain Variable Region eiwitten (IGHV), maken deel uit van de zware
keten in immunoglobulines, ook wel antilichamen genoemd. Samen met de lichte ketens herkennen deze lichaamsvreemde stoffen (antigenen) en helpen zo het immuunsysteem lichaamsvreemde stoffen uit het lichaam te verwijderen. Zowel de zware keten, als de lichte keten zijn telkens opgebouwd uit een onveranderlijk deel (constant region) en een variabel deel (variable
region). De variabele regio heeft enkele hypervariabele delen die zorgen voor de specifieke herkenning van een bepaald antigen, en minder variabele delen die voor de secundaire structuur van het eiwit zorgen.

We hebben enkele eiwitsequenties van de variabele regio’s voor menselijke IGHV’s (Homo
sapiens, n = 15), IGHV’s in muis (Mus musculus, n = 3) en IGHV’s in orang-oetang (Pongo
albelii, n = 3). We zijn ge¨ınteresseerd of deze eiwitten tussen de verschillende species veel
verschillen vertonen. Je vindt de eiwitsequenties terug in **eiwitseq.fa**.

**4.2. Lees de data in**

In [ ]:
url='https://raw.githubusercontent.com/jverwaer/ugain_machine_learning/main/Data/eiwitseq.fa'
df=pd.read_csv(url)
df.head()

**4.3. Bepaal via een paarsgewijze vergelijking de Levenshtein afstand tussen de sequenties doormiddel van het jellyfish package.**

In [ ]:
n=df.shape[0] #aantal datapunten

lev_matrix=np.zeros((***, ***), dtype='int') #alloceer resultaat

for i in range(***):
    for j in range(***):
        lev_matrix[***, ***]=jellyfish.levenshtein_distance(df[***][***], df[***][***])

**4.4. Voer een MDS analyse uit op de Levenshtein afstandsmatrix.**

In [ ]:
model_mds=MDS(dissimilarity=***)
pos=model_mds.fit_transform(lev_matrix)

**4.5. Maak een grafiek waarbij je de eiwit labels meegeeft en de punten kleurt per soort. Wat kan je hier uit besluiten?**

In [ ]:
#plotnine
df_mds=pd.DataFrame()
df_mds['Dim1']=pos[***]
df_mds['Dim2']=pos[***]
df_mds['Soort']=*** #soort


sns.relplot(***)

## 5. ILLUSTRATIE: t-SNE

*****

In de onderstaande sectie wordt het gebruik van t-SNE geïllustreerd.

**Stap 1: inladen data**

In [ ]:
# inlezen data
url='https://raw.githubusercontent.com/jverwaer/ugain_machine_learning/main/Data/iris.csv'

df = pd.read_csv(url, sep = ";")
df.head()

**Stap 2: fit tsne**

In [ ]:
X=np.array(df.loc[:, 'sepal length':'petal width']) #selecteer numerieke data
tsne = TSNE(n_components=2, n_iter=1000)
points = tsne.fit_transform(X)

**Stap 3: vorm om naar dataframe**

In [ ]:
#omvormen naar dataframe
df_tsne=pd.DataFrame()
df_tsne['Dim1']=points[:, 0] #kolom 1
df_tsne['Dim2']=points[:, 1] #kolom 2
df_tsne['Soort']=df['soort'] #soort

**Stap 4: plot coordinaten**

In [ ]:
#met plotnine
sns.relplot(df_tsne,
            x='Dim1',
            y='Dim2',
            hue='Soort')

## 6. ILLUSTRATIE: UMAP

*****

In de onderstaande sectie wordt het gebruik van UMAP (Uniform Manifold Approximation and Projection)  geïllustreerd.

**Stap 1: inladen UMAP en dataset**

In [ ]:
!python -m pip install umap-learn
#er is een andere module met dezelfde naam, dus je moet op deze manier importeren:
import umap.umap_ as umap 

# inlezen data
url='https://raw.githubusercontent.com/jverwaer/ugain_machine_learning/main/Data/iris.csv'

df = pd.read_csv(url, sep = ";")
df.head()

**Stap 2: fit een reducer op de data (cfr. PCA)**

In [ ]:
#via random_state kan je ervoor zorgen dat je 
#altijd hetzelfde resultaat krijgt als je de code opnieuw loopt
iris_array=np.array(df.loc[:, 'sepal length':'petal width']) #selecteer numerieke data
reducer = umap.UMAP(random_state=9000) 
reducer.fit(iris_array)

**Stap 3: gebruik de reduceren om de data te transformeren naar 2D coordinaten**

In [ ]:
embedding = reducer.transform(iris_array)

#omvormen naar dataframe
df_umap=pd.DataFrame()
df_umap['Dim1']=embedding[:, 0] #kolom 1
df_umap['Dim2']=embedding[:, 1] #kolom 2
df_umap['Soort']=df['soort'] #soort

**Stap 4: plot de nieuwe coordinaten**

In [ ]:
#met plotnine
sns.relplot(df_umap,
            x='Dim1',
            y='Dim2',
            hue='Soort')